# 6 Visualise labelled data and remove artefacts

Code that let us overlay each frame of video with outputs from the models. And create time series plots. 

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import ultralytics.utils as ultrautils
import utils
import display
import json

In [ ]:
videos_in = r"..\LookitLaughter.test"
metadata_file = "_LookitLaughter.xlsx"
data_out = r"..\data\1_interim"
videos_out = r"..\data\2_final"

#a couple of files for testing
VIDEO_FILE  = os.path.join(videos_in, "2UWdXP.joke1.rep2.take1.Peekaboo.mp4")
VIDEO_FILE2 = os.path.join(videos_in, "2UWdXP.joke2.rep1.take1.NomNomNom.mp4")
AUDIO_FILE = os.path.join(data_out, "2UWdXP.joke1.rep2.take1.Peekaboo.wav")
SPEECH_FILE = os.path.join(data_out, "2UWdXP.joke1.rep2.take1.Peekaboo.json")

testset = [VIDEO_FILE, VIDEO_FILE2] 

In [ ]:
processedvideos = utils.getprocessedvideos(data_out)
processedvideos.head()

## 6.1 Add annotations onto a video.



In [ ]:
#let's grab a single frame from the video

framenum = 100
video = cv2.VideoCapture(VIDEO_FILE)
video.set(cv2.CAP_PROP_POS_FRAMES, framenum)
ret, frame = video.read()

if ret:
    plt.imshow(frame)
    plt.show()

#look in processed videos to see if we have a keypoints file for this video
videoname = os.path.basename(VIDEO_FILE)
videodata = processedvideos[processedvideos['VideoID'] == videoname]
if videodata.shape[0] > 0:
    print("We have a keypoints file for this video")
    kptsfile = videodata['Keypoints.file'].values[0]
    #Load the keypoints file
    kpts = pd.read_csv(kptsfile)
    kpts.head()

video.release()

In [ ]:
print(f"kptsfile: {kptsfile}")

kpts.iloc[3]

In [ ]:


bboxlabels, bboxes, xycs = utils.getframekpts(kpts, 100)
print(bboxlabels)
print(bboxes)
print(xycs)

In [ ]:
h, w = frame.shape[:2]
print(h,w)    

frame = display.drawOneFrame(frame, bboxlabels, bboxes, xycs,"Peek-a-boo")

plt.imshow(frame)
plt.show()


In [ ]:
with open(SPEECH_FILE) as f:
    speechjson = json.load(f)
caption = display.WhisperExtractCurrentCaption(speechjson, 100, 15)
print(caption)

In [ ]:
width = 640
height = 480
capbox = [[int(width /2), int(0.9*height), int(width/2), int(0.9*height)]]

print(capbox[0][3])

In [ ]:
#look in processed videos to see if we have a keypoints file for this video
videoname = os.path.basename(VIDEO_FILE)
videodata = processedvideos[processedvideos['VideoID'] == videoname]
if videodata.shape[0] > 0:
    kptsfile = videodata['Keypoints.file'].values[0]
    if os.path.exists(kptsfile):
        print(f"Keypoints found: {kptsfile}")
        kpts = pd.read_csv(kptsfile, index_col=None)
    facesfile = videodata['Faces.file'].values[0]
    if os.path.exists(facesfile):
        print(f"Face data found: {facesfile}")
        facedata = pd.read_csv(facesfile, index_col=None)
    speechfile = videodata['Speech.file'].values[0]
    if os.path.exists(speechfile):
        print(f"Speech data found: {speechfile}")
        #open speech json file
        with open(speechfile) as f:
            speechdata = json.load(f)


In this demo, the extract movement algorithm has mislabelled the adult and the child (the labels get applied at random). We need to swap the labels around.

In [ ]:
kpts = utils.relabelPersonIndex(kpts, person = "child", index = 0, newPerson = "temp", newIndex = 100)
kpts = utils.relabelPersonIndex(kpts, person = "adult", index = 1, newPerson = "child", newIndex = 0)
kpts = utils.relabelPersonIndex(kpts, person = "temp", index = 100,newPerson = "adult",  newIndex = 1)   

In [ ]:

annotatedVideo = display.createAnnotatedVideo(VIDEO_FILE, kpts, facedata, speechdata, videos_out)

In [ ]:
#Add the audio file back onto the annotated video
annotatedVideo = r"..\data\2_final\2UWdXP.joke1.rep2.take1.Peekaboo_annotated.mp4"

from moviepy.editor import *
videoclip = VideoFileClip(annotatedVideo)
audioclip = AudioFileClip(AUDIO_FILE)

videoclip = videoclip.set_audio(audioclip)
#note need to save with a different name as the original video is still open
videoclip.write_videofile(r"..\data\2_final\2UWdXP.joke1.rep2.take1.Peekaboo_annotated_audio.mp4")

In [ ]:
videoclip.ipython_display()


## 6.2 TODO Add annotations to all vidoes.

Generate annotated videos for all videos in the test set.

In [ ]:


kpts.head()

# 6.3 TODO Correct person labels in all videos.

Swap parent and child if these are wrong. Ignore other people in video.